In [1]:
import pandas as pd
import numpy as np

In [2]:
cleaned_data=pd.read_csv("actual_cleaned_data.csv")

In [3]:
cleaned_data.head()

,index,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_ID,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME,CUSTOMER_TYPE,CITY,STATE,POSTAL_CODE,item_name,cleaned_item_name,updated_quantity,item_name_no_qty
0,1,362204699,2156,2024-07-24,7247194287,0,1,1,3,7,9,18,10 pc Grilled Wings Combo,41,10,Grilled Wings Combo
1,2,362204699,2156,2024-07-24,7247194287,0,1,1,3,7,9,18,8 pc Grilled Wings Combo,41,8,Grilled Wings Combo
2,3,362204699,2156,2024-07-24,7247194287,0,1,1,3,7,9,18,8 pc Spicy Wings Combo,68,8,Spicy Wings Combo
3,4,269612955,1419,2025-02-15,791214421,0,1,1,3,9,5,1,Ranch Dip - Regular,63,1,Ranch Dip - Regular
4,5,269612955,1419,2025-02-15,791214421,0,1,1,3,9,5,1,50 pc Grilled Wings,40,50,Grilled Wings


In [8]:
small_data=cleaned_data.head(100)

In [10]:
small_data.head(5)

,index,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_ID,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME,CUSTOMER_TYPE,CITY,STATE,POSTAL_CODE,item_name,cleaned_item_name,updated_quantity,item_name_no_qty
0,1,362204699,2156,2024-07-24,7247194287,0,1,1,3,7,9,18,10 pc Grilled Wings Combo,41,10,Grilled Wings Combo
1,2,362204699,2156,2024-07-24,7247194287,0,1,1,3,7,9,18,8 pc Grilled Wings Combo,41,8,Grilled Wings Combo
2,3,362204699,2156,2024-07-24,7247194287,0,1,1,3,7,9,18,8 pc Spicy Wings Combo,68,8,Spicy Wings Combo
3,4,269612955,1419,2025-02-15,791214421,0,1,1,3,9,5,1,Ranch Dip - Regular,63,1,Ranch Dip - Regular
4,5,269612955,1419,2025-02-15,791214421,0,1,1,3,9,5,1,50 pc Grilled Wings,40,50,Grilled Wings


In [11]:
small_data.to_csv("small_data.csv", index=False)

In [12]:
# Create item-order pivot table (item co-occurrence)
basket = cleaned_data.groupby(['ORDER_ID', 'item_name_no_qty'])['updated_quantity'].sum().unstack(fill_value=0)


In [13]:
basket

item_name_no_qty,$.Crispy Feast,Add Grilled Wings,Add Spicy Wings,Blue Cheese Dip - Large,Blue Cheese Dip - Regular,Bottled Beverage,Bottled Cream Soda,Bottled Root Beer,Bottled Sparkling Water,Bottled Water,...,Sub Box,Triple Chocolate Cake,Triple Feast Deal,Unavailable Item,Veggie Sticks,Veggie Sticks Spicy,Voodoo Fries - Large,Voodoo Fries - Regular,Wings Box,Wings Lunch Combo
ORDER_ID,,,,,,,,,,,,,,,,,,,,,
19042,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21421,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49314,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58820,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62930,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999958668,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9999962374,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9999965622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

similarity_matrix = pd.DataFrame(cosine_similarity(basket.T),
                                 index=basket.columns,
                                 columns=basket.columns)


In [15]:
def get_recommendations(cart_items, sim_matrix, top_n=3):
    scores = pd.Series(dtype=float)
    for item in cart_items:
        if item in sim_matrix:
            scores = scores.add(sim_matrix[item], fill_value=0)
    scores = scores.drop(labels=cart_items, errors='ignore')  # remove items already in cart
    return scores.sort_values(ascending=False).head(top_n).index.tolist()


In [16]:
# Example cart
cart = ['Grilled Wings Combo', 'Ranch Dip - Regular']
recommendations = get_recommendations(cart, similarity_matrix)
print(recommendations)


['Spicy Wings Combo', 'Grilled Wings', 'Regular Buffalo Fries']


In [17]:
test_data=pd.read_csv("test_data_question.csv")

In [18]:
import pandas as pd

# Assume test_data is the test set loaded
recommendations_list = []

for _, row in test_data.iterrows():
    cart_items = [row['item1'], row['item2'], row['item3']]  # adjust if you have more items
    recs = get_recommendations(cart_items, similarity_matrix)
    recommendations_list.append(recs)


In [21]:
# Safely pad recommendations to always have 3 items
padded_recommendations = [r + ['NA'] * (3 - len(r)) for r in recommendations_list]

# Assign safely
test_data['RECOMMENDATION 1'] = [r[0] for r in padded_recommendations]
test_data['RECOMMENDATION 2'] = [r[1] for r in padded_recommendations]
test_data['RECOMMENDATION 3'] = [r[2] for r in padded_recommendations]

In [23]:
test_data.to_excel('TeamName_Recommendation Output Sheet.xlsx', index=False)


In [25]:
# Use 'actual_item' or 'missing_item' if it exists
if 'actual_item' in test_data.columns:
    correct = sum(
        test_data.apply(lambda row: row['actual_item'] in 
                        [row['RECOMMENDATION 1'], row['RECOMMENDATION 2'], row['RECOMMENDATION 3']], axis=1)
    )
    recall_at_3 = correct / len(test_data)
    print(f"Recall@3: {recall_at_3:.4f}")


#model 2

### XGBoost

In [28]:
from sklearn.preprocessing import LabelEncoder

# Assume you have item1, item2, item3, and candidate_item in your dataset
le = LabelEncoder()

cleaned_data['item_encoded'] = le.fit_transform(cleaned_data['item_name_no_qty'])



In [30]:
import pandas as pd
import numpy as np

# Original encoded dataset
orders = cleaned_data.groupby('ORDER_ID')['item_encoded'].apply(list).reset_index()
unique_items = cleaned_data['item_encoded'].unique()

training_rows = []

for _, row in orders.iterrows():
    cart = row['item_encoded']
    
    # Positive samples (real items)
    for item in cart:
        training_rows.append({
            'cart_item_1': cart[0] if len(cart) > 0 else -1,
            'cart_item_2': cart[1] if len(cart) > 1 else -1,
            'cart_item_3': cart[2] if len(cart) > 2 else -1,
            'candidate_item': item,
            'label': 1
        })
    
    # Negative samples (random items not in cart)
    neg_items = np.random.choice(list(set(unique_items) - set(cart)), size=3, replace=False)
    for item in neg_items:
        training_rows.append({
            'cart_item_1': cart[0] if len(cart) > 0 else -1,
            'cart_item_2': cart[1] if len(cart) > 1 else -1,
            'cart_item_3': cart[2] if len(cart) > 2 else -1,
            'candidate_item': item,
            'label': 0
        })

train_cleaned_data = pd.DataFrame(training_rows)


In [31]:
train_cleaned_data.head()

,cart_item_1,cart_item_2,cart_item_3,candidate_item,label
0,41,-1,-1,41,1
1,41,-1,-1,16,0
2,41,-1,-1,70,0
3,41,-1,-1,48,0
4,41,-1,-1,41,1


In [34]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Features and labels
X =train_cleaned_data[['cart_item_1', 'cart_item_2', 'cart_item_3', 'candidate_item']]
y =train_cleaned_data['label']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = XGBClassifier(eval_metric='logloss')  # Clean version
model.fit(X_train, y_train)


# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.98      0.97      0.97    847845
           1       0.96      0.96      0.96    571406

    accuracy                           0.97   1419251
   macro avg       0.97      0.97      0.97   1419251
weighted avg       0.97      0.97      0.97   1419251



In [38]:
def recommend_top3(cart_items, model, label_encoder, all_items):
    import numpy as np
    import pandas as pd

    cart1 = cart_items[0] if len(cart_items) > 0 else -1
    cart2 = cart_items[1] if len(cart_items) > 1 else -1
    cart3 = cart_items[2] if len(cart_items) > 2 else -1

    candidates = list(set(all_items) - set(cart_items))

    test_rows = []
    for item in candidates:
        test_rows.append([cart1, cart2, cart3, item])

    # Predict probabilities
    preds = model.predict_proba(test_rows)[:, 1]  # probability of label = 1

    # Pair and sort
    scored = list(zip(candidates, preds))
    top3 = sorted(scored, key=lambda x: x[1], reverse=True)[:3]

    # Return decoded item names
    return label_encoder.inverse_transform([item[0] for item in top3])


In [39]:
submission_rows = []

for _, row in test_data.iterrows():
    # Encode cart items
    cart = [row['item1'], row['item2'], row['item3']]  # or however many you have
    cart_encoded = le.transform([item for item in cart if item in le.classes_])
    
    # Skip if no valid items
    if len(cart_encoded) == 0:
        submission_rows.append(['NA', 'NA', 'NA'])
        continue
    
    # Get top 3 recommendations
    recs = recommend_top3(cart_encoded, model, le, unique_items)
    submission_rows.append(recs)


In [41]:
test_data[['RECOMMENDATION 1', 'RECOMMENDATION 2', 'RECOMMENDATION 3']] = pd.DataFrame(submission_rows)
test_data.to_excel("DataForge_Recommendation Output Sheet.xlsx", index=False)


In [43]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

# Define the hyperparameter search space
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [0.5, 1.0, 1.5]
}

# Instantiate XGBoost model
xgb = XGBClassifier(eval_metric='logloss', tree_method='hist')

# Use RandomizedSearchCV with single-threaded mode to avoid pickling issues
search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=10,             # Try 10 different parameter combinations
    scoring='f1',          # Use 'recall' or 'roc_auc' if preferred
    cv=3,                  # 3-fold cross-validation
    verbose=1,
    n_jobs=1               # Single-threaded to avoid PicklingError
)

# Fit the model
search.fit(X_train, y_train)

# Print best params and optionally use the best model
print("✅ Best Parameters Found:")
print(search.best_params_)

best_model = search.best_estimator_

# Evaluate on test set
from sklearn.metrics import classification_report
y_pred = best_model.predict(X_test)
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 10 candidates, totalling 30 fits
✅ Best Parameters Found:
{'subsample': 1.0, 'reg_lambda': 0.5, 'reg_alpha': 0.01, 'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 0.8}

📊 Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.97    847845
           1       0.96      0.96      0.96    571406

    accuracy                           0.97   1419251
   macro avg       0.97      0.97      0.97   1419251
weighted avg       0.97      0.97      0.97   1419251

